In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

import os
import warnings


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/'My Drive'/UniversalEmb

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/UniversalEmb


In [3]:
%cd  sst2-splited

/content/drive/My Drive/UniversalEmb/sst2-splited


In [0]:
!ls

In [0]:
from glob import glob
files = glob('./neg/*')

In [29]:
len(files)

12500

In [0]:
sent = {'review':[], 'lbl':[]}

In [0]:
from tqdm import tqdm
for file_ in tqdm(files):
  g = open(file_).read()
  sent['review'].append(g)

In [32]:
len(sent['review'])

9935

In [0]:
imdb_data = pd.read_csv('movie_review.csv')

In [14]:
imdb_data

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos
...,...,...,...,...,...,...
64715,9,cv999,14636,20,that lack of inspiration can be traced back to...,neg
64716,9,cv999,14636,21,like too many of the skits on the current inca...,neg
64717,9,cv999,14636,22,"after watching one of the "" roxbury "" skits on...",neg
64718,9,cv999,14636,23,"bump unsuspecting women , and . . . that's all .",neg


In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
imdb_data = pd.DataFrame(sent['review'])

In [0]:
imdb_data.columns = ['review']

In [0]:
#Tokenization of text
tokenizer=ToktokTokenizer()
#Setting English stopwords
stopword_list=nltk.corpus.stopwords.words('english')

In [0]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text
#Apply function on review column
imdb_data['review']=imdb_data['review'].apply(denoise_text)

In [0]:
from nltk.tokenize import sent_tokenize
f = open("neg_sentence_prep.txt", 'w')
for sentences in imdb_data['review'].tolist():
  for sentence in sent_tokenize(sentences):
    f.write(sentence+'\n')
f.close()

In [11]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
!head SentimentPolarityDatasetVersion2_pos_prepared_sentences.txt

A typic romp through cheech and chong' realiti which includ drugs, singing, more drugs, car and driving, even more drugs, pee wee, aliens, gasoline, laundry, stand up comedy, surprisingli more drug and space coke !!.
It is not as coher or plausibl as Up in smoke but it still is incred funny, without becom as strang as nice dreams.
there are some classic scenes, which includ the open scene where they get some ga for their car and the drive to work.
also funni is cheech' song (mexican-americans) and chong' follow up song.
anoth notabl scene is the welfar offic scene with jone (human nois machine), from the polic academi series, and the old laugh man.
all in all, thi is a great follow up to Up in smoke and is quit watchabl when sober or not.-celluloid rehab
jonathan demme' directori debut for roger corman' legendari exploit outfit new world pictur rate highli as one of the finest chicks-in-chain 70' grindhous classic to ever grace celluloid.
beauteou russ meyer starlet eric ("vixen," "bey

In [0]:
!wc -l SentimentPolarityDatasetVersion2_pos_prepared_sentences.txt

47410 SentimentPolarityDatasetVersion2_pos_prepared_sentences.txt
